In [57]:
from __future__ import annotations

import functools as ft

import pandas as pd

In [42]:
urls = pd.read_csv("../data/processed/urls.csv", index_col=0).dropna().reset_index(drop=True)

In [43]:
def decode_url(url: str) -> str:
    try:
        return bytearray(url, "utf-8").decode("idna")
    except Exception:
        return url
    
    
urls["url"] = urls["url_host"].apply(decode_url)
urls_ = urls[["url"]].drop_duplicates().reset_index(drop=True)

In [44]:
def split_domains(url: str) -> list[str]:
    try:
        return url.split(".")
    except Exception:
        return url


urls_["domains"] = urls_["url"].apply(split_domains)

In [52]:
urls_["domains"].apply(len).value_counts()

2    122326
3     74804
4      1751
1       490
5       152
6         2
7         1
Name: domains, dtype: int64

In [53]:
urls_[urls_["domains"].apply(len) >= 6]

,url,domains
4206,wp1.pachka89.13296.spectrum.myjino.ru,"[wp1, pachka89, 13296, spectrum, myjino, ru]"
92171,static.familyguy.tinyco.com.s3.amazonaws.com,"[static, familyguy, tinyco, com, s3, amazonaws..."
134349,wp1.lexiqq.mzdxp.spectrum.myjino.ru,"[wp1, lexiqq, mzdxp, spectrum, myjino, ru]"


In [54]:
urls_["domains"].apply(len).value_counts(normalize=True).sort_index().cumsum()

1    0.002456
2    0.615539
3    0.990447
4    0.999223
5    0.999985
6    0.999995
7    1.000000
Name: domains, dtype: float64

In [71]:
urls_["domains"].explode().nunique()

171335

In [72]:
urls_["domains"].apply(lambda d: d[-4:] if len(d) > 4 else d).explode().nunique()

171198

In [73]:
urls_["domain1"] = urls_["domains"].apply(lambda d: d[-1])
urls_["domain2"] = urls_["domains"].apply(lambda d: d[-2] if len(d) > 2 else None)
urls_["domain3"] = urls_["domains"].apply(lambda d: d[-3] if len(d) > 3 else None)